 # **Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np
import pymysql  

## Extract - Load Datasets

In [29]:


files = {
    '2020': "D:/Gartner/IT_Services_Marketshare_Masked/IT_Services_Marketshare_2020 (742642).CSV",
    '2020Q1': "D:/Gartner/IT_Services_Marketshare_Masked/IT_Services_Marketshare_2020Q1.CSV",
    '2021': "D:/Gartner/IT_Services_Marketshare_Masked/IT_Services_Marketshare_2021 (765402).CSV",
    '2022': "D:/Gartner/IT_Services_Marketshare_Masked/IT_Services_Marketshare_2022 (787876).CSV",
    '2023': "D:/Gartner/IT_Services_Marketshare_Masked/IT_Services_Marketshare_2023 (808454).CSV"
}


## Load datasets

In [30]:

datasets = {year: pd.read_csv(file) for year, file in files.items()}

## Display sample data

In [33]:
for year, df in datasets.items():
    print(f"Dataset {year}:\n", df.head(), "\n")

Dataset 2020:
       Year    Super Region          Region         Country    Vendor  \
0  2019 YR  Eastern Europe  Eastern Europe  Czech Republic  Vendor 2   
1  2019 YR  Eastern Europe  Eastern Europe  Czech Republic  Vendor 2   
2  2019 YR  Eastern Europe  Eastern Europe  Czech Republic  Vendor 2   
3  2019 YR  Eastern Europe  Eastern Europe  Czech Republic  Vendor 2   
4  2019 YR  Eastern Europe  Eastern Europe  Czech Republic  Vendor 2   

     Service 1                                        Service 2  \
0  IT Services  Application Implementation and Managed Services   
1  IT Services  Application Implementation and Managed Services   
2  IT Services  Application Implementation and Managed Services   
3  IT Services  Application Implementation and Managed Services   
4  IT Services  Application Implementation and Managed Services   

                    Service 3                          Vertical    Ticker  \
0  Application Implementation              Banking & Securities  Ticker 

# **Tranfformation**

## Standardizing column names

In [38]:
for year, df in datasets.items():
    df.columns = df.columns.str.strip().str.replace("Vendor.1", "Vendor").str.replace(" & ", " and ")
    datasets[year] = df


## Clean & Merge Datasets
 Standardizing column names

In [39]:
for year, df in datasets.items():
    df.columns = df.columns.str.strip()  
    df.columns = df.columns.str.replace(" ", "_") 
    df.columns = df.columns.str.replace("Super_Region", "SuperRegion") 
    df.columns = df.columns.str.replace("Vendor.1", "Vendor")
    df.columns = df.columns.str.replace("ConstantCurrency_Revenue-USD", "ConstantCurrencyRevenueUSD")

# If SuperRegion is missing (like in 2023 dataset), add a placeholder
    if "SuperRegion" not in df.columns:
        df["SuperRegion"] = "Unknown"

    datasets[year] = df  # Update dataset after modifications


# Fixing Vendor Column in 2021 & 2022

## Replace generic Vendor column with actual vendor names for 2021 and 2022

In [40]:
for year in ["2021", "2022"]:
    if "Vendor.1" in datasets[year].columns:
        datasets[year]["Vendor"] = datasets[year]["Vendor.1"]
        datasets[year].drop(columns=["Vendor.1"], inplace=True)


# Cleaning Year Column
## Extracting the numeric year from 'Year' column

In [41]:
for year, df in datasets.items():
    df["Year"] = df["Year"].astype(str).str.extract(r"(\d{4})").astype(int)
    datasets[year] = df

# Ensuring Numeric Data Types for Revenue
## Convert revenue columns to numeric

In [42]:
for year, df in datasets.items():
    df["VendorRevenue-USD"] = pd.to_numeric(df["VendorRevenue-USD"], errors="coerce")
    df["ConstantCurrencyRevenueUSD"] = pd.to_numeric(df["ConstantCurrencyRevenueUSD"], errors="coerce")
    datasets[year] = df


# Ensure Unique Columns and Align Structures
## Before merging, let's identify if any dataset has duplicate columns.

In [45]:
for year, df in datasets.items():
    duplicate_cols = df.columns[df.columns.duplicated()].tolist()
    if duplicate_cols:
        print(f"Dataset '{year}' has duplicate columns: {duplicate_cols}")


Dataset '2021' has duplicate columns: ['vendor']
Dataset '2022' has duplicate columns: ['vendor']


# Remove or Rename Duplicate Columns

In [46]:
for year, df in datasets.items():
    if "vendor" in df.columns[df.columns.duplicated()].tolist():
        print(f"Dataset '{year}' has duplicate 'vendor' column. Checking values:")
        print(df.filter(like="vendor").head())


Dataset '2021' has duplicate 'vendor' column. Checking values:
     vendor     vendor  vendorrevenue-usd
0  Vendor 2  Accenture           5.432218
1  Vendor 2  Accenture           8.562894
2  Vendor 2  Accenture           0.098677
3  Vendor 2  Accenture           4.811882
4  Vendor 2  Accenture           5.020963
Dataset '2022' has duplicate 'vendor' column. Checking values:
     vendor     vendor  vendorrevenue-usd
0  Vendor 2  Accenture           7.055103
1  Vendor 2  Accenture          10.720365
2  Vendor 2  Accenture           0.142403
3  Vendor 2  Accenture           6.334292
4  Vendor 2  Accenture           6.392021


# Rename One Column (If Different Data Exists)

In [47]:
for year, df in datasets.items():
    vendor_cols = [col for col in df.columns if "vendor" in col]
    if len(vendor_cols) > 1: 
        df.rename(columns={vendor_cols[1]: "vendor_alt"}, inplace=True)
        datasets[year] = df


# remove duplicate

In [49]:
for year, df in datasets.items():
    df = df.loc[:, ~df.columns.duplicated()] 
    datasets[year] = df


# Merge all datasets into a single DataFrame
## Reset index to avoid duplicate index issues

In [50]:

final_df = pd.concat([df.reset_index(drop=True) for df in datasets.values()], ignore_index=True)


# Final Data output

In [51]:
print(final_df)

         year     superregion              region         country      vendor  \
0        2019  Eastern Europe      Eastern Europe  Czech Republic    Vendor 2   
1        2019  Eastern Europe      Eastern Europe  Czech Republic    Vendor 2   
2        2019  Eastern Europe      Eastern Europe  Czech Republic    Vendor 2   
3        2019  Eastern Europe      Eastern Europe  Czech Republic    Vendor 2   
4        2019  Eastern Europe      Eastern Europe  Czech Republic    Vendor 2   
...       ...             ...                 ...             ...         ...   
1702451  2023         Unknown  Sub-Saharan Africa    South Africa  Vendor 194   
1702452  2023         Unknown  Sub-Saharan Africa    South Africa  Vendor 194   
1702453  2023         Unknown  Sub-Saharan Africa    South Africa  Vendor 194   
1702454  2023         Unknown  Sub-Saharan Africa    South Africa  Vendor 194   
1702455  2023         Unknown  Sub-Saharan Africa    South Africa  Vendor 194   

           service_1       

# Handling Missing Values
## Fill missing values for key columns
## Standardize column names (lowercase + strip spaces)

In [52]:
final_df.columns = final_df.columns.str.lower().str.strip()

# Now fill missing values
final_df["superregion"].fillna("Unknown", inplace=True)
final_df["region"].fillna("Unknown", inplace=True)
final_df["vendor"].fillna("Unknown", inplace=True)
final_df["vertical"].fillna("Other", inplace=True)



C:\Users\ssuun\AppData\Local\Temp\ipykernel_9104\2364875693.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df["superregion"].fillna("Unknown", inplace=True)
C:\Users\ssuun\AppData\Local\Temp\ipykernel_9104\2364875693.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

 # List all columns to check for typos

In [53]:
print(final_df.columns) 


Index(['year', 'superregion', 'region', 'country', 'vendor', 'service_1',
       'service_2', 'service_3', 'vertical', 'ticker', 'hq_country',
       'vendor_alt', 'constantcurrencyrevenueusd', 'vendorrevenue-usd'],
      dtype='object')


In [54]:
print("Columns in final_df:", final_df.columns.tolist())


Columns in final_df: ['year', 'superregion', 'region', 'country', 'vendor', 'service_1', 'service_2', 'service_3', 'vertical', 'ticker', 'hq_country', 'vendor_alt', 'constantcurrencyrevenueusd', 'vendorrevenue-usd']


# Check Missing Values (NaN Counts)

In [55]:
print(final_df.isnull().sum())


year                               0
superregion                        0
region                             0
country                            0
vendor                             0
service_1                          0
service_2                          0
service_3                          1
vertical                           0
ticker                             2
hq_country                     22268
vendor_alt                         1
constantcurrencyrevenueusd         2
vendorrevenue-usd             914876
dtype: int64


# Check Data Types

In [56]:
print(final_df.dtypes)

year                            int64
superregion                    object
region                         object
country                        object
vendor                         object
service_1                      object
service_2                      object
service_3                      object
vertical                       object
ticker                         object
hq_country                     object
vendor_alt                     object
constantcurrencyrevenueusd    float64
vendorrevenue-usd             float64
dtype: object


# Again Handling missing value

In [59]:
# Fill missing values for categorical columns
final_df["service_3"].fillna("Unknown", inplace=True)
final_df["ticker"].fillna("Unknown", inplace=True)
final_df["hq_country"].fillna("Unknown", inplace=True)
final_df["vendor_alt"].fillna("Unknown", inplace=True)

# Fill missing values for numerical columns
final_df["constantcurrencyrevenueusd"].fillna(final_df["constantcurrencyrevenueusd"].median(), inplace=True)

# Handle `vendorrevenue-usd`
if final_df["vendorrevenue-usd"].isnull().sum() > len(final_df) * 0.5:
    final_df.drop(columns=["vendorrevenue-usd"], inplace=True)  # Drop column if >50% missing
else:
    final_df["vendorrevenue-usd"].fillna(final_df["vendorrevenue-usd"].median(), inplace=True)  # Fill if needed


C:\Users\ssuun\AppData\Local\Temp\ipykernel_9104\975621959.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df["service_3"].fillna("Unknown", inplace=True)
C:\Users\ssuun\AppData\Local\Temp\ipykernel_9104\975621959.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

# Again Check Missing Values (NaN Counts)

In [60]:
print(final_df.isnull().sum())

year                          0
superregion                   0
region                        0
country                       0
vendor                        0
service_1                     0
service_2                     0
service_3                     0
vertical                      0
ticker                        0
hq_country                    0
vendor_alt                    0
constantcurrencyrevenueusd    0
dtype: int64


# Again check for duplicate rows

In [61]:
print(final_df.duplicated().sum())  # Count duplicate rows

48544


# Remove duplicates

In [62]:
final_df = final_df.drop_duplicates().reset_index(drop=True)

# Sum of Duplicates

In [63]:
print(final_df.duplicated().sum())

0


# Check Data Types Again

In [65]:
print(final_df.dtypes)

year                            int64
superregion                    object
region                         object
country                        object
vendor                         object
service_1                      object
service_2                      object
service_3                      object
vertical                       object
ticker                         object
hq_country                     object
vendor_alt                     object
constantcurrencyrevenueusd    float64
dtype: object


# Save the Cleaned Data 

In [66]:
final_df.to_csv("IT_Services_Cleaned_Data.csv", index=False)


# Renaming Some column for understanding

In [ ]:
import pandas as pd
file_path = "IT_Services_Cleaned_Data.csv" 
df = pd.read_csv(file_path)

# Rename the column
df.rename(columns={"vendor_alt": "vendorrevenue_usd"}, inplace=True)
# df.rename(columns={"vendorrevenue_usd": "vendor_revenue_usd"}, inplace=True)
#df.rename(columns={"constantcurrencyrevenueusd": "constant_currency_revenue_usd"}, inplace=True)
# Save the updated file (overwrite original or create a new file)
df.to_csv("IT_Services_Cleaned_Data.csv", index=False)  # Overwrite original


# **Load Data To MySQL**

# Install MySQL Connector

In [3]:

!pip install mysql-connector-python


# Verify MySQL Connector Installation

In [2]:
import mysql.connector
print("MySQL Connector is installed and working!")


MySQL Connector is installed and working!


# Connection to MySQL

In [3]:
import mysql.connector

try:
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Sun!l@123",
        database="it_service_market_share_db"
    )

    if conn.is_connected():
        print("Connected to MySQL successfully!")

    conn.close()

except mysql.connector.Error as err:
    print(f"Error: {err}")


Connected to MySQL successfully!


In [69]:
import sys
print(sys.executable)


C:\Users\ssuun\AppData\Local\Programs\Python\Python313\python.exe


# Load the Cleaned Data into MySQL

In [112]:
import pandas as pd
df = pd.read_csv("IT_Services_Cleaned_Data.csv")

df.head()

C:\Users\ssuun\AppData\Local\Temp\ipykernel_9104\1679947916.py:11: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("IT_Services_Cleaned_Data.csv")


,year,superregion,region,country,vendor,service_1,service_2,service_3,vertical,ticker,hq_country,vendor_revenue_usd,constant_currency_revenue_usd
0,2019,Eastern Europe,Eastern Europe,Czech Republic,Vendor 2,IT Services,Application Implementation and Managed Services,Application Implementation,Banking & Securities,Ticker 2,United States,5.693025,5.632195
1,2019,Eastern Europe,Eastern Europe,Czech Republic,Vendor 2,IT Services,Application Implementation and Managed Services,Application Implementation,"Communications, Media & Services",Ticker 2,United States,8.248331,8.160199
2,2019,Eastern Europe,Eastern Europe,Czech Republic,Vendor 2,IT Services,Application Implementation and Managed Services,Application Implementation,Education,Ticker 2,United States,0.098793,0.097737
3,2019,Eastern Europe,Eastern Europe,Czech Republic,Vendor 2,IT Services,Application Implementation and Managed Services,Application Implementation,Government,Ticker 2,United States,4.37442,4.327679
4,2019,Eastern Europe,Eastern Europe,Czech Republic,Vendor 2,IT Services,Application Implementation and Managed Services,Application Implementation,Healthcare Providers,Ticker 2,United States,2.630113,2.602010


# Insert Data from Jupyter to MySQL

In [127]:
import mysql.connector

# Establish MySQL connection
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Sun!l@123",
    database="it_service_market_share_db"
)
cursor = conn.cursor()

# SQL query to insert data
insert_query = """
INSERT INTO it_services_market (year, superregion, region, country, vendor, service_1, service_2, service_3, 
                         vertical, ticker, hq_country,  vendor_revenue_usd, constant_currency_revenue_usd)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insert each row from DataFrame
for i, row in df.iterrows():
    cursor.execute(insert_query, tuple(row))

# Commit changes and close connection
conn.commit()
conn.close()

print("✅ Data successfully inserted into MySQL!")


✅ Data successfully inserted into MySQL!
